#Managed MCP servers in Databricks

MCP servers are like connectors that allow AI agents to use outside data and tools. With Databricks, you don’t need to build these connections yourself — the managed MCP servers let your agents quickly plug into Unity Catalog data, vector search, and custom functions. Please refer in our [documentation page](https://docs.databricks.com/aws/en/generative-ai/mcp/managed-mcp) to find out more about managed mcp servers available in Databricks.


## 0. Get started with the Genie MCP managed servers in Databricks

In this accelerator, we’ll work with a healthcare dataset to show how agents and Genie as a Databricks managed MCP server can be applied in practice. We’ll use [Genie](https://docs.databricks.com/aws/en/genie/) inside Databricks to explore the data, ask questions in natural language, and quickly generate insights. This example illustrates how MCP-powered agents can help domain experts interact with complex datasets without writing from scratch these capabilities.




## 1. Create a Genie space in Databricks

In order to get started first you need to create your Genie space in your Databricks Workspace. To setup your Genie workspace, refer to the [documentation](https://docs.databricks.com/aws/en/genie/set-up#-create-a-genie-space). To make sure your Genie space is setup with the best practices, have a look [here](https://docs.databricks.com/gcp/en/genie/best-practices). In our example we have setup our genie space as below:<br><br>

<img src="/files/images/screenshot.png" style="float: right" width="800px">



## 2. Use the Genie MCP server in your agent code. 

In Databricks you can build agents in the framework of your choice. In our example we have used LangGraph to build the agent in the Databricks [Mosaic AI framework](https://docs.databricks.com/aws/en/generative-ai/agent-framework/create-agent) and expose the Genie capabilities from the managed MCP server. For the managed Databricks MCP servers, you would just need to put the specific managed MCP server url, in our case we would need to put the url of the Genie space we have created. The code below is just a snippet of the complete code, with the main change you need to do. This is following already the publicly available [notebook example](https://docs.databricks.com/aws/en/generative-ai/mcp/managed-mcp#notebooks-build-an-agent-with-managed-mcp-servers) which you could adjust it in your case as well. To see how the complete full code works in action, check the [mcp-tool-call-agent](url) notebook.


In [0]:
import asyncio
import mlflow
import os
import json
from uuid import uuid4
from pydantic import BaseModel, create_model
from typing import Annotated, Any, Generator, List, Optional, Sequence, TypedDict, Union

from databricks_langchain import (
    ChatDatabricks,
    UCFunctionToolkit,
    VectorSearchRetrieverTool,
)
from databricks_mcp import DatabricksOAuthClientProvider, DatabricksMCPClient
from databricks.sdk import WorkspaceClient

from langchain_core.language_models import LanguageModelLike
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langchain_core.messages import (
    AIMessage,
    AIMessageChunk,
    BaseMessage,
    convert_to_openai_messages,
)
from langchain_core.tools import BaseTool, tool

from langgraph.graph import END, StateGraph
from langgraph.graph.graph import CompiledGraph
from langgraph.graph.message import add_messages
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt.tool_node import ToolNode

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client as connect

from mlflow.entities import SpanType
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
)

import nest_asyncio
nest_asyncio.apply()

############################################
## Define your LLM endpoint and system prompt
############################################
# TODO: Replace with your model serving endpoint
LLM_ENDPOINT_NAME = "databricks-claude-3-7-sonnet"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)

# TODO: Update with your system prompt
system_prompt = """You are a healthcare data assistant. Your main tasks are:

1. When asked to explain, summarize, or analyze healthcare datasets, use the Databricks Genie tools to provide insights, data explanations, and context."""

###############################################################################
## Configure MCP Servers for your agent
## This section sets up server connections so your agent can retrieve data or take actions.
###############################################################################

# TODO: Choose your MCP server connection type.

# ----- Simple: Managed MCP Server (no extra setup required) -----
# Uses your Databricks Workspace settings and Personal Access Token (PAT) auth.
workspace_client = WorkspaceClient()

# Managed MCP Servers: Ready to use with default settings above
host = workspace_client.config.host
MANAGED_MCP_SERVER_URLS = [
    # TODO: Add here your genie space id
    f"{host}/api/2.0/mcp/genie/01f076c46ca8181ba15b996ff7462de2",
]

Test the Databricks Genie MCP server directly in the Databricks [AI Playground](https://docs.databricks.com/aws/en/large-language-models/ai-playground#use-ai-playground). You would just need to add your Genie space and then start asking questions using the model endpoint of your choice. In future, you'd be able to also export the notebook code from the playground as normally is possible for non-MCP agentic applications. Additionally, you could also expose the Genie capabilities to external client application such as Claude Desktop. To see how is this done, you could follow the instructions explained [here](https://github.com/alexxx-db/databricks-genie-mcp).

<img src="/files/images/playground-1.png" style="float: right">


Then start asking questions.

<img src="/files/images/playground-2.png" style="float: right">


Additionally, you could also inspect the View Trace option after each response to see how is the tool calling being made. </br></br> 

<img src="/files/images/playground-3.png" style="float: right">


## UC functions as tools or MCP server tools?

You might be wondering when you would be using [UC functions](https://docs.databricks.com/aws/en/udf/unity-catalog) or using Databricks managed MCP servers. Both of them being very important to provide capabilities to your agents, UC function would be defintely recommend if you know in advance what functions your agent needs and you are building them complelety in databricks, this way bring all the governance benefits that UC funtions powered by Unity Catalog bring. However if you building a multi-agentic system and you need to share the capabilities across either coming from Databricks Managed MCP servers or you need to do some additional external API calls (custom MCPs) then exposing these as tools in a MCP server would be very efficient for your agentic system.